# Hotelrain: Querying & Populatin the ontology

## rdflib lets us work with rdf files but also use SPARQL queries

In [2]:
import rdflib
from rdflib import Graph

In [38]:
g=Graph()

g.parse("Hotelrain - RDF.rdf")

http://www.owl-ontologies.com/unnamed.owl#Aix-en-Provence TGV does not look like a valid URI, trying to serialize this will break.
http://www.owl-ontologies.com/unnamed.owl#Avignon TGV does not look like a valid URI, trying to serialize this will break.
http://www.owl-ontologies.com/unnamed.owl#Champagne-Ardenne TGV does not look like a valid URI, trying to serialize this will break.
http://www.owl-ontologies.com/unnamed.owl#La Rochelle-Ville does not look like a valid URI, trying to serialize this will break.
http://www.owl-ontologies.com/unnamed.owl#Valence TGV does not look like a valid URI, trying to serialize this will break.
http://www.owl-ontologies.com/unnamed.owl#Meuse TGV does not look like a valid URI, trying to serialize this will break.
http://www.owl-ontologies.com/unnamed.owl#La Teste does not look like a valid URI, trying to serialize this will break.
http://www.owl-ontologies.com/unnamed.owl#TGV Haute-Picardie does not look like a valid URI, trying to serialize this wi

<Graph identifier=N4074dbf82ac44e44b9c46053a29e4e57 (<class 'rdflib.graph.Graph'>)>

### Typing the query :

In [34]:
query_test="""
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX ns: <http://www.owl-ontologies.com/unnamed.owl#>
PREFIX cim: <http://iec.ch/TC57/2010/CIM-schema-cim15#>

SELECT * WHERE {
  ?poi rdf:type ns:Traveler.
  ?poi ns:hasName ?Name.
  ?poi ns:hasAge ?Age.
 FILTER(?Age>22)
} 
"""

qres=g.query(query_test)

In [35]:
for row in qres:
    print(f"{row.Age}")

32


## Populating the ontology by reading csv files

### Hotels & Campings

In [2]:
import pandas as pd

In [4]:
df=pd.read_csv("hebergements-classes.csv",sep=";")
#df.drop(df.iloc[:,:1],axis=1,inplace=True)

#### Here are all the columns we will need :
- coordonnees
- NOM COMMERCIAL
- TELEPHONE
- SITE INTERNET
- CLASSEMENT
- Nom du département

In [9]:
df2=df[["coordonnees","NOM COMMERCIAL","TELEPHONE","SITE INTERNET","CLASSEMENT","Nom du département"]].copy()
df2.dropna(inplace=True)
df2 = df2[df2.CLASSEMENT != 'Aire naturelle']

In [10]:

lat=[]
long=[]

for i in df2["coordonnees"]:
    coor=i.split(',')
    lat.append(coor[0])
    long.append(coor[1])

In [11]:
stars=[]
for g in df2["CLASSEMENT"]:
    #print(int(g[0]))
    stars.append(int(g[0]))

#### Hotel individual example

In [139]:

individuals=""
for index in range(len(lat)):
    
    #''.join(e for e in df2["NOM COMMERCIAL"].loc[index] if e.isalnum())
    try:
        hotel=f"""<owl:NamedIndividual rdf:about="http://www.owl-ontologies.com/unnamed.owl#{''.join(e for e in df2["NOM COMMERCIAL"].loc[index] if e.isalnum())}{index}">
        <rdf:type rdf:resource="http://www.owl-ontologies.com/unnamed.owl#Hotel"/>
        <hasDistrict rdf:datatype="http://www.w3.org/2001/XMLSchema#string">{df2["Nom du département"].loc[index]}</hasDistrict>
        <hasLatitude rdf:datatype="http://www.w3.org/2001/XMLSchema#float">{lat[index]}</hasLatitude>
        <hasLink rdf:datatype="http://www.w3.org/2001/XMLSchema#string">{df2["SITE INTERNET"].loc[index].replace("&","")}</hasLink>
        <hasLongitude rdf:datatype="http://www.w3.org/2001/XMLSchema#float">{long[index]}</hasLongitude>
        <hasName rdf:datatype="http://www.w3.org/2001/XMLSchema#string">{''.join(e for e in df2["NOM COMMERCIAL"].loc[index] if e.isalnum())}</hasName>
        <hasNumber rdf:datatype="http://www.w3.org/2001/XMLSchema#string">{df2["TELEPHONE"].loc[index]}</hasNumber>
        <hasStar rdf:datatype="http://www.w3.org/2001/XMLSchema#int">{stars[index]}</hasStar>
    </owl:NamedIndividual>\n\n"""
    except KeyError:
        pass
        
        
    
    individuals+=hotel

In [140]:
#df2["Nom du département"].unique()

individuals
with open("indivs.txt","w") as f:
    f.write(individuals)

### Creating Trips

In [6]:
dft=pd.read_csv("garesnet.csv")

In [7]:
lst=list(dft["Nom_Gare"].values)


In [8]:
import random

random.shuffle(lst)


#### Trip individual example

In [39]:
allTrips=""

for i in range(50):
    
    
    owl_trip=f"""
     <owl:NamedIndividual rdf:about="http://www.owl-ontologies.com/unnamed.owl#Trip{i+i}">
        <rdf:type rdf:resource="http://www.owl-ontologies.com/unnamed.owl#Trip"/>
        <hasArrival rdf:resource="http://www.owl-ontologies.com/unnamed.owl#{lst.pop()}"/>
        <hasDeparture rdf:resource="http://www.owl-ontologies.com/unnamed.owl#{lst.pop()}"/>
        <hasTraveler rdf:resource="http://www.owl-ontologies.com/unnamed.owl#I{random.randint(1,4)}"/>
    </owl:NamedIndividual>\n\n
    
    """
    allTrips+=owl_trip

In [40]:
with open("trips.rdf","w") as f:
    f.write(allTrips)